In [2]:
#带加密
import math
import time
import numpy as np
from phe import paillier
import pandas as pd
from sklearn import datasets
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics  import accuracy_score, f1_score

In [7]:
pk, sk = paillier.generate_paillier_keypair()
p1=pk.encrypt(10)
p2 = pk.encrypt(20)
P1 = p1+p2
P2 = 20*p1
P1 = p1 + p2 + 2*10*(p2)

print("P1:{}".format(sk.decrypt(P1)))
print("P2:{}".format(sk.decrypt(P2)))



P1:430
P2:200


In [3]:
class Client:
    def __init__(self, config):
        ## 模型参数
        self.config = config
        ## 中间计算结果
        self.data = {}
        ## 与其他节点的连接状况
        self.other_client = {}
    
    ## 与其他参与方建立连接
    def connect(self, client_name, target_client):
        self.other_client[client_name] = target_client
    
    ## 向特定参与方发送数据
    def send_data(self, data, target_client):
        target_client.data.update(data)

In [ ]:
#辅助节点
class ClientC(Client):
    """
    Client C as trusted dealer.
    """
    def __init__(self, A_d_shape, B_d_shape, config):
        super().__init__(config)
        self.A_data_shape = A_d_shape
        self.B_data_shape = B_d_shape
        self.public_key = None
        self.private_key = None
        ## 保存训练中的损失值（泰展开近似）
        self.loss = []

    #生成Paillier的密钥对
    def task_1(self, client_A_name, client_B_name):
        try:
            public_key, private_key = paillier.generate_paillier_keypair()
            self.public_key = public_key
            self.private_key = private_key
        except Exception as e:
            print("C step 1 error 1: %s" % e)

        data_to_AB = {"public_key": public_key}
        self.send_data(data_to_AB, self.other_client[client_A_name])
        self.send_data(data_to_AB, self.other_client[client_B_name])
        return
    
    #解密由A,B发来的加密梯度和loss,step4
    def task_2(self,client_A_name,client_B_name):
        dt = self.data
        assert "encrypted_loss" in dt.keys(), "Error: 'encrypted_loss' from A in step 4 not successfully received."
        assert "encrypted_loss" in dt.keys(), "Error: 'encrypted_loss' from A in step 4 not successfully received."

In [ ]:
#主动方
class ClientA(Client):
    def __init__(self, X, y, config):
        super().__init__(config)
        self.X = X
        self.y = y
        self.weights = np.zeros(X.shape[1])
        
    def compute_z_a(self):
        z_a = np.dot(self.X, self.weights)
        return z_a
    
    
	## 加密梯度的计算，对应step4
    def compute_encrypted_dJ_a(self, encrypted_u):
        encrypted_dJ_a = self.X.T.dot(encrypted_u) + self.config['lambda'] * self.weights
        return encrypted_dJ_a
    
	##参数的更新
    def update_weight(self, dJ_a):
        self.weights = self.weights - self.config["lr"] * dJ_a / len(self.X)
        return
    
    #step2
    def task_1(self,client_B_name,client_C_name):
        try:
            dt = self.data
            assert "public_key" in dt.keys(), "Error: 'public_key' from C in step 1 not successfully received."
            pk = dt['public_key']
        except Exception as e:
            print("A step 1 exception: %s" % e)
        try:
            z_a = self.compute_z_a()
            z_a_square = z_a ** 2
            encrypted_z_a = np.asarray([pk.encrypt(x) for x in z_a])
            encrypted_z_a_square = np.asarray([pk.encrypt(x) for x in z_a_square])
            dt.update({"z_a": z_a})
        except Exception as e:
            print("Wrong 1 in A: %s" % e)

        ##计算加密loss，loss为了决定啥时候停止训练
        encrypted_z_b = dt["encrypted_z_b"]
        encrypted_z_b_square = dt["encrypted_z_b_square"]

        enctyted_z = encrypted_z_a + encrypted_z_b
        encrypted_z_square = encrypted_z_b_square + encrypted_z_a_square + 2*z_a*(encrypted_z_b)
        
        encrypted_loss = np.sum(0.125*encrypted_z_square-0.5*self.y*enctyted_z)     #其他项与训练无关，为了简化省去其他项

        ##计算残差项d，算梯度
        encrypted_d = 0.25 * enctyted_z - 0.5 * pk.encrypt(self.y)

        dt.update({"encrypted_loss":encrypted_loss,"encrypted_d": encrypted_d})
        
        #计算自己的梯度
        encrypted_gradient_A = self.X.T.dot(encrypted_d) + self.config['lambda'] * self.weights

        data_to_C = {"encrypted_loss":encrypted_loss,"encrypted_gradient_A":encrypted_gradient_A}
        self.send_data(data_to_C,client_C_name)

        data_to_B = {"encrypted_d":encrypted_d}
        self.send_data(data_to_B,client_B_name)

    



In [ ]:
#参与方
class ClientB(Client):
    def __init__(self, X, config):
        super().__init__(config)
        self.X = X
        self.weights = np.zeros(X.shape[1])
        self.data = {}
        
    # def compute_u_b(self):
    #     z_b = self.compute_z_b()
    #     u_b = 0.25 * z_b - 0.5 * self.y
    #     return u_b
    
    def compute_z_b(self):
        z_b = np.dot(self.X, self.weights)  
        return z_b

    def compute_encrypted_dJ_b(self, encrypted_u):      #计算b的加密梯度
        encrypted_dJ_b = self.X.T.dot(encrypted_u) + self.config['lambda'] * self.weights
        return encrypted_dJ_b

    def update_weight(self, dJ_b):
        self.weights = self.weights - self.config["lr"] * dJ_b / len(self.X)

    #step1
    def task_1(self,client_A_name):
        dt = self.data
        assert "public_key" in dt.keys(), "Error: 'public_key' from C in step 1 not successfully received."
        pk = dt["public_key"]
        z_b = self.compute_z_b()
        z_b_square = z_b ** 2
        try:
            encrypted_z_b = np.asarray([pk.encrypt(x) for x in z_b])
            encrypted_z_b_square = np.asarray([pk.encrypt(x) for x in z_b_square])
        except Exception as e:
            print("Encypt fail, Wrong 1 in B: %s" % e)
        dt.update({"encrypted_z_b": encrypted_z_b})
        data_to_A = {"encrypted_z_b": encrypted_z_b,"encrypted_z_b_square":encrypted_z_b_square}
        self.send_data(data_to_A,self.other_client[client_A_name])

    #step3
    def task_2(self,client_C_name):
        dt = self.data
        assert "encrypted_d" in dt.keys(), "Error: 'encrypted_d' from A in step3 not successfully received."

        #计算自己的梯度
        encrypted_d = dt["encrypted_d"]
        encrypted_gradient_B = self.X.T.dot(encrypted_d) + self.config['lambda'] * self.weights

        data_to_C = {"encrypted_gradient_B":encrypted_gradient_B}

        self.send_data(data_to_C,client_C_name)

        
        